In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch_geometric as pyg
import torch_geometric.datasets as pygd
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [5]:
from gpn.experiments.multiple_run_experiment import MultipleRunExperiment
from gpn.experiments.transductive_experiment import TransductiveExperiment
from gpn.utils.config import (
    DataConfiguration,
    ModelConfiguration,
    RunConfiguration,
    TrainingConfiguration,
)


def create_experiment(model_name):
    run_cfg = RunConfiguration(
        job="train",
        eval_mode="default",
        experiment_directory="./.cache",
        save_model=True,
        gpu=0,
        experiment_name="ood_loc",
        num_inits=1,
        num_splits=1,
    )
    data_cfg = DataConfiguration(
        dataset="AmazonPhotos",
        split_no=1,
        root="./data",
        ood_flag=True,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )

    model_cfg = ModelConfiguration(
        model_name=model_name,
        seed=42,
        init_no=1,
        dim_hidden=64,
        dropout_prob=0.5,
        K=1,
        add_self_loops=True,
        maf_layers=0,
        gaussian_layers=0,
        use_batched_flow=True,
        loss_reduction="sum",
        approximate_reg=True,
        flow_weight_decay=0.0,
        pre_train_mode="flow",
        alpha_evidence_scale="latent-new",
        alpha_teleport=0.1,
        entropy_reg=0.0001,
        dim_latent=16,
        radial_layers=10,
    )

    train_cfg = TrainingConfiguration(
        epochs=100000,
        stopping_mode="default",
        stopping_patience=50,
        stopping_restore_best=True,
        stopping_metric="val_CE",
        stopping_minimize=True,
        finetune_epochs=0,
        warmup_epochs=5,
        lr=0.01,
        weight_decay=0.001,
    )

    return MultipleRunExperiment(run_cfg, data_cfg, model_cfg, train_cfg)


# gpn_e = create_experiment("GPN")
lop_e = create_experiment("GPN_LOP")
res_lop = lop_e.run()
# res = gpn_e.run()
res_lop

Epoch 1/5:
w SparseTensor(row=tensor([   0,    0,    0,  ..., 7649, 7649, 7649], device='cuda:0'),
             col=tensor([   0, 1210, 2198,  ..., 3295, 4196, 7649], device='cuda:0'),
             val=tensor([0.1818, 0.0170, 0.0072,  ..., 0.0472, 0.1006, 0.3250], device='cuda:0'),
             size=(7650, 7650), nnz=245812, density=0.42%)
e tensor([721.2976, 708.0585, 726.5742,  ..., 736.7337, 722.9227, 730.2520],
       device='cuda:0', grad_fn=<SumBackward1>)


RuntimeError: mat.dim() >= 2 INTERNAL ASSERT FAILED at "csrc/cuda/spmm_cuda.cu":110, please report a bug to PyTorch. Input mismatch

In [ ]:
import pandas as pd
from gpn.utils.utils import results_dict_to_df

df_lop = results_dict_to_df(res_lop)
df = results_dict_to_df(res)

pd.set_option("display.max_rows", None)
(pd.DataFrame(
    dict(
        val_gpn=df["val"],
        test_gpn=df["test"],
        val_lop=df_lop["val"],
        test_lop=df_lop["test"],
    )
)).round(4).dropna()

In [ ]:
from gpn.data.dataset_manager import DatasetManager
from gpn.data.dataset_provider import InMemoryDatasetProvider
import torch_sparse as ts

ds = InMemoryDatasetProvider(
    DatasetManager(
        dataset="AmazonPhotos",
        split_no=1,
        root="./data",
        ood_flag=True,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )
).to_sparse()

batch = list(ds.loader())[0]

batch.adj_t.dtype()

In [ ]:
N = batch.x.shape[0]
adj_t = batch.adj_t.cuda()
adj_t

In [16]:
import torch_sparse as ts

adj_t = ts.SparseTensor.from_dense(
    torch.Tensor([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
).cuda()
x = torch.Tensor([1, 10, 100]).view(-1,1).cuda()
N = x.shape[0]
adj_t.to_dense()

tensor([[0., 1., 0.],
        [1., 0., 1.],
        [0., 1., 0.]], device='cuda:0')

In [29]:
from gpn.layers.appnp_propagation import APPNPPropagation

prop = APPNPPropagation(1, 0, 0, normalization="in-degree")
# x = torch.diag(torch.ones(N, device="cuda"))
x = ts.SparseTensor.eye(N, dtype=torch.float32, device="cuda")  # type: ignore

prop(x, adj_t).to_dense()

tensor([[0.5000, 0.3333, 0.0000],
        [0.5000, 0.3333, 0.5000],
        [0.0000, 0.3333, 0.5000]], device='cuda:0')

In [ ]:
ts.SparseTensor.from_edge_index(torch.tensor([[0,1],[0,1]]),sparse_sizes=(10,3)).dtype()